# Cleaning Demographic Data

To see the features selected to be included in the cleaned demographic data, scroll to the bottom of this notebook to where the dictionary "meta_features" is printed. The cleaned data contains those features, which are written to a csv file named "cleaned_demographic_features.csv" in the data folder.

In [14]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [21]:
demCensus=pd.read_csv('data/allDemographics2010.csv')
demCensus['GEO.id2'] = demCensus['GEO.id2'].astype(str)
demCensus['GEO.id2'] = [(5-len(zipcode))*'0' + zipcode if len(zipcode) < 5 else zipcode for zipcode in demCensus['GEO.id2']]
demCensus.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_S001,HD02_S001,HD01_S002,HD02_S002,HD01_S003,HD02_S003,HD01_S004,HD02_S004,HD01_S005,HD02_S005,HD01_S006,HD02_S006,HD01_S007,HD02_S007,HD01_S008,HD02_S008,HD01_S009,HD02_S009,HD01_S010,HD02_S010,HD01_S011,HD02_S011,HD01_S012,HD02_S012,HD01_S013,HD02_S013,HD01_S014,HD02_S014,HD01_S015,HD02_S015,HD01_S016,HD02_S016,HD01_S017,HD02_S017,HD01_S018,HD02_S018,HD01_S019,HD02_S019,HD01_S020,HD02_S020,HD01_S021,HD02_S021,HD01_S022,HD02_S022,HD01_S023,HD02_S023,HD01_S024,...,HD01_S162,HD02_S162,HD01_S163,HD02_S163,HD01_S164,HD02_S164,HD01_S165,HD02_S165,HD01_S166,HD02_S166,HD01_S167,HD02_S167,HD01_S168,HD02_S168,HD01_S169,HD02_S169,HD01_S170,HD02_S170,HD01_S171,HD02_S171,HD01_S172,HD02_S172,HD01_S173,HD02_S173,HD01_S174,HD02_S174,HD01_S175,HD02_S175,HD01_S176,HD02_S176,HD01_S177,HD02_S177,HD01_S178,HD02_S178,HD01_S179,HD02_S179,HD01_S180,HD02_S180,HD01_S181,HD02_S181,HD01_S182,HD02_S182,HD01_S183,HD02_S183,HD01_S184,HD02_S184,HD01_S185,HD02_S185,HD01_S186,HD02_S186
0,Id,00Id2,Geography,Number; SEX AND AGE - Total population,Percent; SEX AND AGE - Total population,Number; SEX AND AGE - Total population - Under...,Percent; SEX AND AGE - Total population - Unde...,Number; SEX AND AGE - Total population - 5 to ...,Percent; SEX AND AGE - Total population - 5 to...,Number; SEX AND AGE - Total population - 10 to...,Percent; SEX AND AGE - Total population - 10 t...,Number; SEX AND AGE - Total population - 15 to...,Percent; SEX AND AGE - Total population - 15 t...,Number; SEX AND AGE - Total population - 20 to...,Percent; SEX AND AGE - Total population - 20 t...,Number; SEX AND AGE - Total population - 25 to...,Percent; SEX AND AGE - Total population - 25 t...,Number; SEX AND AGE - Total population - 30 to...,Percent; SEX AND AGE - Total population - 30 t...,Number; SEX AND AGE - Total population - 35 to...,Percent; SEX AND AGE - Total population - 35 t...,Number; SEX AND AGE - Total population - 40 to...,Percent; SEX AND AGE - Total population - 40 t...,Number; SEX AND AGE - Total population - 45 to...,Percent; SEX AND AGE - Total population - 45 t...,Number; SEX AND AGE - Total population - 50 to...,Percent; SEX AND AGE - Total population - 50 t...,Number; SEX AND AGE - Total population - 55 to...,Percent; SEX AND AGE - Total population - 55 t...,Number; SEX AND AGE - Total population - 60 to...,Percent; SEX AND AGE - Total population - 60 t...,Number; SEX AND AGE - Total population - 65 to...,Percent; SEX AND AGE - Total population - 65 t...,Number; SEX AND AGE - Total population - 70 to...,Percent; SEX AND AGE - Total population - 70 t...,Number; SEX AND AGE - Total population - 75 to...,Percent; SEX AND AGE - Total population - 75 t...,Number; SEX AND AGE - Total population - 80 to...,Percent; SEX AND AGE - Total population - 80 t...,Number; SEX AND AGE - Total population - 85 ye...,Percent; SEX AND AGE - Total population - 85 y...,Number; SEX AND AGE - Total population - Media...,Percent; SEX AND AGE - Total population - Medi...,Number; SEX AND AGE - Total population - 16 ye...,Percent; SEX AND AGE - Total population - 16 y...,Number; SEX AND AGE - Total population - 18 ye...,Percent; SEX AND AGE - Total population - 18 y...,Number; SEX AND AGE - Total population - 21 ye...,Percent; SEX AND AGE - Total population - 21 y...,Number; SEX AND AGE - Total population - 62 ye...,...,Number; HOUSEHOLDS BY TYPE - Total households ...,Percent; HOUSEHOLDS BY TYPE - Total households...,Number; HOUSEHOLDS BY TYPE - Total households ...,Percent; HOUSEHOLDS BY TYPE - Total households...,Number; HOUSEHOLDS BY TYPE - Total households ...,Percent; HOUSEHOLDS BY TYPE - Total households...,Number; HOUSEHOLDS BY TYPE - Total households ...,Percent; HOUSEHOLDS BY TYPE - Total households...,Number; HOUSEHOLDS BY TYPE - Total households ...,Percent; HOUSEHOLDS BY TYPE - Total households...,Number; HOUSEHOLDS BY TYPE - Total households ...,Percent; HOUSEHOLDS BY TYPE - Total households...,Number; HOUSEHOLDS BY TYPE - Total hou

In [22]:
demCensus.shape

(33121, 375)

In [17]:
#exploring the different features in this table
prelimFeatures = demCensus.loc[0]
for i in prelimFeatures:
    print i

Id
00Id2
Geography
Number; Total population
Percent; Total population
Number; Total population - SEX AND AGE - Male
Percent; Total population - SEX AND AGE - Male
Number; Total population - SEX AND AGE - Female
Percent; Total population - SEX AND AGE - Female
Number; Total population - SEX AND AGE - Under 5 years
Percent; Total population - SEX AND AGE - Under 5 years
Number; Total population - SEX AND AGE - 5 to 9 years
Percent; Total population - SEX AND AGE - 5 to 9 years
Number; Total population - SEX AND AGE - 10 to 14 years
Percent; Total population - SEX AND AGE - 10 to 14 years
Number; Total population - SEX AND AGE - 15 to 19 years
Percent; Total population - SEX AND AGE - 15 to 19 years
Number; Total population - SEX AND AGE - 20 to 24 years
Percent; Total population - SEX AND AGE - 20 to 24 years
Number; Total population - SEX AND AGE - 25 to 34 years
Percent; Total population - SEX AND AGE - 25 to 34 years
Number; Total population - SEX AND AGE - 35 to 44 years
Percent; Tot

In [20]:
#keep these non-percentage data points: id, zipcode, raw population numbers
#the rest of the cleaned data will be percentages
features = demCensus.filter(regex='GEO').ix[1:, :]
features["A001"] = demCensus["HD01_S001"][1:].astype(float)

#start a supplementary table of metadata
meta_features = {"A001": "Number; SEX AND AGE - Total population"}

features.head()

,GEO.id,GEO.id2,GEO.display-label,A001
1,8600000US00601,00601,ZCTA5 00601,19143
2,8600000US00602,00602,ZCTA5 00602,42042
3,8600000US00603,00603,ZCTA5 00603,55530
4,8600000US00604,00604,ZCTA5 00604,3923
5,8600000US00606,00606,ZCTA5 00606,6449


In [ ]:
#Clean up all strings and convert to float
#Replace '(X)' strings with NaN
#Get rid of the first column of 100% values 

percent_features_num = demCensus.filter(regex='HD02').ix[1:, 1:].replace(to_replace='(X)', value=np.nan, regex=True)
totalpop_agegroups = percent_features_num.astype(float)
percent_features = demCensus.filter(regex='HD02').ix[:0, 1:].append(totalpop_agegroups)

raw_features = demCensus.filter(regex='HD01').ix[1:, 1:].replace(to_replace='(X)', value=np.nan, regex=True)\
    .astype(float)

In [ ]:
percent_features.head()

In [ ]:
#function for aggregating data

def aggregate_data(start, end, interval, df, larger_df, category, gender, lst):
    #create temporary dataframe with correct length
    new_df = demCensus.filter(regex='HD02').ix[1:, :1]
    new_df = new_df.drop("HD02_S001", 1)
    length = len(df.columns)
    
    #aggregate two columns to get percentages for age groups of 10 years
    for i in range(start, end, interval):    
        first_col = 'HD02_S0' + ('%02d' % (i+2))
        second_col = 'HD02_S0' + ('%02d' % (i+3))
        col_name = "A0" + ('%02d' % ((i-start)/interval + length - 2))
        new_df[col_name] = larger_df[first_col] + larger_df[second_col]
    
    #update feature names
    for i in range(len(lst)):
        col_name = "A0" + ('%02d' % (i+length-2))
        meta_features[col_name] = "Percent; " + category + " - " + gender + " population - " + lst[i]
    
    #update external dataframe with results (this is cumulative)
    for column in new_df:
        df[column] = new_df[column]

    return df

In [ ]:
#get aggregated age data: do this section as a whole because these steps are cumulative

#set up parameters
lst = ["Under 10 years", 
       "10 to 19 years", 
       "20 to 29 years", 
       "30 to 39 years", 
       "40 to 49 years", 
       "50 to 59 years", 
       "60 to 69 years", 
       "70 to 79 years", 
       "Over 80 years"]

#getting data for total population
df = aggregate_data(0, 18, 2, features, percent_features, "SEX AND AGE", "Total", lst)

#getting data for male population
next_colname = "A0" + ('%02d' % (len(df.columns)-2))
df[next_colname] = percent_features["HD02_S026"] 
meta_features[next_colname] = percent_features["HD02_S026"][0]
df2 = aggregate_data(25, 43, 2, df, percent_features, "SEX AND AGE", "Male", lst)

#getting data for female population
next_colname = "A0" + ('%02d' % (len(df.columns)-2))
df2[next_colname] = percent_features["HD02_S051"] 
meta_features[next_colname] = percent_features["HD02_S051"][0]
df3 = aggregate_data(50, 68, 2, df2, percent_features, "SEX AND AGE", "Female", lst)

df3.head() #this dataframe has aggregated age data

In [ ]:
#get aggregated race data: also do this section as a whole because these steps are cumulative

#get White, Black or African American, American Indian and Alaska Native and Asian race data
for i in range(0,4):
    next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
    df3[next_colname] = percent_features["HD02_S0" + str(78 + i)][1:] 
    meta_features[next_colname] = percent_features["HD02_S0" + str(78 + i)][0]

#get Hawaiian and other Pacific Islander data
next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
df3[next_colname] = percent_features["HD02_S089"][1:] 
meta_features[next_colname] = percent_features["HD02_S089"][0]

#get Hispanic data
next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
df3[next_colname] = percent_features["HD02_S107"][1:]
meta_features[next_colname] = percent_features["HD02_S107"][0]

df3.head() #this dataframe now has aggregated age and race data

In [ ]:
#get household data
#can update housing features here if we want to add more from the original data
lst = ["HD02_S151", "HD02_S155", "HD02_S156", "HD02_S157", "HD02_S158", "HD02_S160", 
       "HD02_S161", "HD02_S162", "HD02_S163", "HD02_S164", "HD02_S165",
       "HD02_S166", "HD02_S170", "HD02_S171", "HD02_S172", "HD02_S174"]
lst_raw = ["HD01_S167", "HD01_S168", "HD01_S169", "HD01_S186"]

#percentage data points
for i in lst:
    next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
    df3[next_colname] = percent_features[i][1:] 
    meta_features[next_colname] = percent_features[i][0]

#raw data points (e.g. average household size)
for i in lst_raw:
    next_colname = "A0" + ('%02d' % (len(df3.columns) - 2))
    df3[next_colname] = raw_features[i] 
    meta_features[next_colname] = demCensus[i][0]
    
df3.head()

In [ ]:
#see all the feature names for the dataframe
for i in sorted(meta_features):
    print i, meta_features[i]

In [ ]:
df3['GEO.id2']=df3['GEO.id2'].astype(str)
df3.to_pickle("data/cleaned_demographic_features_2010.pkl")